<a href="https://colab.research.google.com/github/PiotrMaciejKowalski/BigData2022-actors/blob/pobieranie_zdj%C4%99%C4%87/pobieranie_zdj%C4%99%C4%87_z_URL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyspark py4j
!pip install -q findspark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.tgz
!tar xf spark-3.3.1-bin-hadoop2.tgz
import pyspark
import findspark
from pyspark.sql import SparkSession
import os
import pyspark.sql.types as T
from pyspark.sql.functions import split, col, monotonically_increasing_id, when
from pyspark.sql.types import StructType, StringType, IntegerType, BooleanType, FloatType, TimestampType, DateType, ArrayType, MapType, StructField
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop2"
spark=SparkSession.builder.appName('Colab').getOrCreate()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 41 kB/s 
     |████████████████████████████████| 200 kB 66.9 MB/s 
     |████████████████████████████████| 199 kB 12.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845512 sha256=a090a5307f031237acf9e257dad63e050671f6d8498ec05cff10c42faa918c07
  Stored in directory: /root/.cache/pip/wheels/43/dc/11/ec201cd671da62fa9c5cc77078235e40722170ceba231d7598
Successfully built pyspark
--2022-12-17 17:02:16--  https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop2.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 274099817 (261M) [application/x-gzip]
Saving to: ‘spark-3.3.1-bin-hadoop2.tgz’

spark-3.3.1-bin-had 100%[===============

In [14]:
from google.colab import drive

In [2]:
!pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import requests

In [21]:
def download_image(url, file_name):
    response = requests.get(url)
    if response.status_code == 200:
        with open(file_name, "wb") as f:
            f.write(response.content)
    else:
        print(response.status_code)

In [5]:
"""def df_download_images(dataframe, num_col, name_string):
  #num_col- numer kolumny, od 0
  for i in range(dataframe.count()):
    download_image(dataframe.collect()[i][num_col], name_string+str(i)+".jpg")"""

In [18]:
def df_download_images(dataframe, num_col, num_namecol):
#num_namecol - numer kolumny, która zawiera nazwy jakie chcemy nadać obrazom (prawdopodobnie numer kolumny zawierającej id aktora)
#num_col - numer kolumny, od 0
  for i in range(dataframe.count()):
    download_image(dataframe.collect()[i][num_col], str(dataframe.collect()[i][num_namecol])+".jpg")

test:

In [22]:
dataframe = [("Lucas","https://images.unsplash.com/photo-1508921912186-1d1a45ebb3c1?ixlib=rb-4.0.3&ixid=MnwxMjA3fDB8MHxzZWFyY2h8Mnx8cGhvdG98ZW58MHx8MHx8&w=1000&q=80","Smith","36636","M",3000),
    ("Elijah","https://www.paperlessmovement.com/wp-content/uploads/2019/09/o2dvsv2pnhe.jpg","","40288","M",4000),
    ("Emma","https://mymodernmet.com/wp/wp-content/uploads/2019/12/photos-of-indonesia-rarindra-prakarsa-4.jpg","Williams","42114","F",4000),
    ("Sophia","https://live-production.wcms.abc-cdn.net.au/50749be1153e1907d7e1208fc96432f8?impolicy=wcms_crop_resize&cropH=844&cropW=1500&xPos=0&yPos=0&width=862&height=485","Jones","39192","F",4000),
    ("Mia","https://media.wired.com/photos/62ccbecc4847c5414f1e3dc9/3:2/w_1280%2Cc_limit/Light-Photo-Video-Like-a-Pro-Gear-GettyImages-142009824.jpg","Brown","","F",-1)
  ]

schema = StructType([ \
    StructField("name",StringType(),True), \
    StructField("url",StringType(),True), \
    StructField("lastname",StringType(),True), \
    StructField("id", StringType(), True), \
    StructField("gender", StringType(), True), \
    StructField("salary", IntegerType(), True) \
  ])
 
df = spark.createDataFrame(data=dataframe,schema=schema)

In [23]:
df_download_images(df, 1, 0)

In [15]:
from PIL import Image
import os
from pathlib import Path